# Post Process para Buckeye
Este script procesa la base de datos de Buckeye y la convierte a una tabla. La tabla actuamente tiene los siguientes campos:
token: la palabra o fonema
duracion: duracion de la palabra o fonema
path: fichero de donde se ha extraido
type: w (word) o p (phonema)
numtoken: una etiqueta para saber que fonemas pertenecen a que palabra

Falta por hacer:
Extraer energia
Contador numero de frases. Decidir que es una frase



In [ ]:
import os
import re
import glob
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def importar_align_words(path):
    """
    Buscar fichero words en un path
    """
    fichero_words = glob.glob(path + "*.words")
    if len(fichero_words) == 1:
        #print("ok")
        pass
    else:
        sys.exit("Error importar fichero words")
        
    words_align = pd.read_csv(fichero_words[0], sep = ' 122 ' , skiprows=9, header=None,
                              names=["t_end", "token"], engine='python')

    df = pd.DataFrame(words_align.token.str.split(';').tolist(),
                      columns = ["token", "phoneme1", "phoneme2", "phoneme3"])
    words_align = pd.concat([words_align.t_end.reset_index(drop=True),df.reset_index(drop=True)], axis=1)
    return(words_align)


def importar_sentences(path):
    """
    Buscar fichero words en un path
    """
    sentences = glob.glob(path + "*.txt")
    if len(sentences) == 1:
        #print("ok")
        pass
    else:
        sys.exit("Error importar fichero words")
        
    sentences_align = pd.read_csv(sentences[0], sep = ' 122 ' , skiprows=9, header=None,
                              names=["t_end", "token"], engine='python')

    df = pd.DataFrame(words_align.token.str.split(';').tolist(),
                      columns = ["token", "phoneme1", "phoneme2", "phoneme3"])
    words_align = pd.concat([words_align.t_end.reset_index(drop=True),df.reset_index(drop=True)], axis=1)
    return(words_align)



def importar_align_phoneme(path):
    """
    Buscar fichero words en un path
    """
    fichero_words = glob.glob(path + "*.phones")
    if len(fichero_words) == 1:
        #print("ok")
        pass
    else:
        sys.exit("Error importar fichero words")
        
    phones_align = pd.read_csv(fichero_words[0], sep = '  ' , skiprows=9, header=None, names=["t_end", "token"], engine='python')
    df = pd.DataFrame(phones_align.token.str.split(' ', 1).tolist(), columns = ["chunk", "token"])
    phones_align = pd.concat([phones_align.t_end.reset_index(drop=True),df.reset_index(drop=True)], axis=1)
    
    return(phones_align)

In [ ]:

# Obtener todas las subcarpetas
folders = [(root[2::] + "/" ) for root, dirs, files in os.walk('.') if not dirs and bool(re.match('^[sabcdfe0-9_/.-]*$', root))]
folders.sort()


In [ ]:
"""
# Convertir fichero en frases uno de los folders unicamente
sentences_path = glob.glob(folders[0] + "*.txt")
sentence_raw = pd.read_csv(sentences_path[0], skiprows=0, header=None, names=["sentence"], engine='python')

sentences  = sentence_raw["sentence"].str.split("<SIL>", expand = True).stack()
sentences = sentences.str.replace(r"\<.*\>", '') # Regular expresion remove all comments
sentences
"""

In [ ]:
"""
a = sentences.reset_index()
# Remove Empty lines
a[0].replace('', np.nan, inplace=True)
a[0].replace(' ', np.nan, inplace=True)
a.dropna(subset=[0], inplace=True)
a = a.reset_index()
a  = a[0].str.split(" ", expand = False)
"""

In [ ]:
num_token = 0
lista_token = []
lista_tinit = []
lista_tend = []
lista_duration = []
lista_numphonemes = []
lista_numletters = []
lista_sentence = []

lista_file = []
lista_type = [] # word or phoneme
lista_numtoken = []

sentence_flag = 0

for path in folders:
    print(path)
    
    # Tabla phonemas y palabras
    words_align = importar_align_words(path)
    phones_align = importar_align_phoneme(path)
    
    
    # Procesamos los datos 
    for index, row in words_align.iterrows():
        # Si hay un silencio (SIL) o habla en entrevistador (IVER) or aspiraciones (VOCNOISE), cambio de frase
        if (row.token == "<SIL>") or (row.token == "<IVER>") or (row.token == "<VOCNOISE>"):
            sentence_flag += 1

        if ((row.token.startswith("<") == False) and (row.token.endswith(">") == False) and 
            (row.token.startswith("{") == False) and (row.token.endswith("}") == False) and
            (row.phoneme2 != None) and (row.phoneme2.strip().split(" ") != [""])):
            num_token += 1
            
            ########################################################################################
            # WORDS ################################################################################
            lista_numtoken.append(num_token)
            lista_token.append(row.token)
            lista_file.append(path)
            lista_type.append("w")
            lista_numphonemes.append(len(row.phoneme2.strip().split(" ")))
            lista_numletters.append(len(row.token))
            lista_sentence.append(sentence_flag)

            
            
            # Duracion word
            lista_tinit.append(words_align.loc[index-1].t_end)
            lista_tend.append(words_align.loc[index].t_end)
            lista_duration.append(words_align.loc[index].t_end - words_align.loc[index-1].t_end) #Fin del actual - Fin del anterior
            
            # Energia word
            
            
            # Frase
            
            
            #######################################################################################
            # ALINEAMIENTO PHONEMAS ###############################################################
            ########################################################################################
            phonemes = row.phoneme2.strip().split(" ") # Convierte fonemas en lista
            if phonemes[0] == 'SIL': # Si han colado un silencio lo eliminamos (BUG corregido)
                phonemes.pop(0)
            
            
            # Search primer fonema
            # BUGS: No coincide el tiempo exacto. Se busca el mas cercano
            # No coincide el phonema exactamente, se filtra
            idx_init = (np.abs(phones_align.t_end - words_align.loc[index-1].t_end)).idxmin() +1
            if (phones_align.loc[idx_init].token.split(";")[0].strip().split("+1")[0] != phonemes[0]): # En caso de no coincidir
                tempdf = (np.abs(phones_align.t_end - words_align.loc[index-1].t_end)).nsmallest(6)
                idx_init = phones_align.loc[tempdf.index].token[phones_align.loc[tempdf.index].token.str.split(" ;", expand=True)[0] == phonemes[0]].index[0]
                print("exception1")

            # Search ultimo fonema
            idx_end = (np.abs(phones_align.t_end - words_align.loc[index].t_end)).idxmin()
            if phones_align.loc[idx_end].token == None:
                idx_end = idx_end-1
            if (phones_align.loc[idx_end].token.split(";")[0].strip().split("+1")[0] != phonemes[-1]): # En caso de no coincidir
                tempdf = (np.abs(phones_align.t_end - words_align.loc[index].t_end)).nsmallest(4)
                idx_end = phones_align.loc[tempdf.index].token[phones_align.loc[tempdf.index].token.str.split(" ;", expand=True)[0] == phonemes[-1]].index[0]
                print("exception2")
                
            if ((phones_align.loc[idx_init].token.split(";")[0].strip().split("+1")[0] == phonemes[0]) & 
                (phones_align.loc[idx_end].token.split(";")[0].strip().split("+1")[0] == phonemes[-1])):
                for idp in range(idx_init, idx_end+1):
                    pass
                    #print(phones_align.loc[idp].token)
            else:
                sys.exit("Error buscando fonemas de la palabra")
            
            # MEDIDAS FONEMA ####################################################
            for idp in range(idx_init, idx_end+1):
                # A veces hay marcas vacias
                if phones_align.loc[idp].token != None:
                    lista_numtoken.append(num_token)
                    phontoken = phones_align.loc[idp].token.split(";")[0].strip().split("+1")[0]
                    lista_token.append(phontoken)
                    lista_file.append(path)
                    lista_sentence.append(sentence_flag)

                    lista_type.append("p")
                    lista_numphonemes.append(1)
                    if phones_align.loc[idp].token != None:
                        lista_numletters.append(len(phones_align.loc[idp].token))
                    else:
                        lista_numletters.append(0)
                        

                
                    # DURACION DEL FONEMA
                    lista_tinit.append(phones_align.loc[idp-1].t_end)
                    lista_tend.append(phones_align.loc[idp].t_end)
                    lista_duration.append(phones_align.loc[idp].t_end - phones_align.loc[idp-1].t_end) #Fin del actual - Fin del anterior

            
            
    # Each time we save the data
    tablaResumen = pd.DataFrame({"token":lista_token, "tinit":lista_tinit, "tend":lista_tend, 
                                "duration":lista_duration, "sentence": lista_sentence, "path": lista_file,
                                "type": lista_type, "numtoken": lista_numtoken, "numphonemes":lista_numphonemes,
                                "numletters": lista_numletters})
    tablaResumen.to_csv("TablaTranscripcion.csv", index = False)